Взял питоновский датасет от сюда
https://huggingface.co/datasets/bigcode/the-stack/tree/main/data/python
Первые 2000 записей


In [1]:
import pyarrow.parquet as pq
import json

In [2]:

# Открываем Parquet файл для чтения
parquet_file = pq.ParquetFile('dataset/train-00000-of-00206.parquet')



In [3]:
columns = parquet_file.schema.names
columns

['hexsha',
 'size',
 'ext',
 'lang',
 'max_stars_repo_path',
 'max_stars_repo_name',
 'max_stars_repo_head_hexsha',
 'item',
 'max_stars_count',
 'max_stars_repo_stars_event_min_datetime',
 'max_stars_repo_stars_event_max_datetime',
 'max_issues_repo_path',
 'max_issues_repo_name',
 'max_issues_repo_head_hexsha',
 'item',
 'max_issues_count',
 'max_issues_repo_issues_event_min_datetime',
 'max_issues_repo_issues_event_max_datetime',
 'max_forks_repo_path',
 'max_forks_repo_name',
 'max_forks_repo_head_hexsha',
 'item',
 'max_forks_count',
 'max_forks_repo_forks_event_min_datetime',
 'max_forks_repo_forks_event_max_datetime',
 'content',
 'avg_line_length',
 'max_line_length',
 'alphanum_fraction']

In [4]:

# Получаем список колонок в файле
columns = parquet_file.schema.names

# Читаем только выбранные колонки ('content' и 'size')
columns_to_read = ['content', 'size']
table = parquet_file.read(columns=columns_to_read)

# Преобразуем данные в pandas DataFrame
df = table.to_pandas()



In [5]:

print(len(df.iloc[0, 0]))
print(df.iloc[0, 1])



6442
6442


In [6]:


df = df[df['size'] < 100_000]

nrows = 2 * 1000
batch_size = 1000

# Разбиваем DataFrame на пакеты по batch_size строк
for i in range(0, nrows, batch_size):
    # Получаем пакет данных
    batch_df = df.iloc[i:i+batch_size]
    
    # Преобразуем DataFrame в список словарей
    data = batch_df.to_dict(orient='records')
    
    # Создаем имя файла для текущего пакета данных
    filename = f'dataset/dataset_{i//batch_size}.json'
    
    # Сериализуем данные в JSON и записываем в файл
    with open(filename, 'w') as json_file:
        json.dump(data, json_file)
        
    print(f'Записан файл {filename} с {len(batch_df)} строками')


Записан файл dataset/dataset_0.json с 1000 строками
Записан файл dataset/dataset_1.json с 1000 строками
